In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import kagglehub
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
import math

tf.random.set_seed(42)
np.random.seed(42)
# Check for GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Download the ReducedMNIST dataset
path = kagglehub.dataset_download("mohamedgamal07/reduced-mnist")
print(f"Dataset downloaded to: {path}")

# List the files in the dataset
files = os.listdir(path)
print("Files in the dataset:", files)

path = path+"/Reduced MNIST Data"

train_dir = path +'/Reduced Trainging data'
test_dir = path+'/Reduced Testing data'



IMG_SIZE = 28
NOISE_DIM = 100
BATCH_SIZE = 128
EPOCHS = 200

TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Dataset downloaded to: /home/youssef-abuzeid/.cache/kagglehub/datasets/mohamedgamal07/reduced-mnist/versions/1
Files in the dataset: ['Reduced MNIST Data']


In [8]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_gen = data_gen.flow_from_directory(train_dir,
                                          target_size=(IMG_SIZE, IMG_SIZE),
                                          color_mode='grayscale',
                                          class_mode='sparse',
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)
test_gen = data_gen.flow_from_directory(test_dir,
                                            target_size=(IMG_SIZE, IMG_SIZE),
                                            color_mode='grayscale',
                                            class_mode='sparse',
                                            batch_size=BATCH_SIZE,
                                            shuffle=False)

                                            
                                          

Found 10000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


## Define the GAN Architecture


### First we'll define the Generator

In [21]:
# Generator Class

class Generator(tf.keras.Model):
    def __init__(self, noise_dim=NOISE_DIM):
        super(Generator, self).__init__()
        self.noise_dim = noise_dim

        self.model = tf.keras.Sequential([
            # Foundation for 7x7 feature maps
            tf.keras.layers.Dense(7*7*256, use_bias=False, input_shape=(self.noise_dim,)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(), 
            tf.keras.layers.Reshape((7, 7, 256)),
            
            # First upsampling block
            tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            
            # Second upsampling block
            tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            
            # Output layer with tanh activation
            tf.keras.layers.Conv2D(1, (5, 5), padding='same', use_bias=True, activation='tanh')
        ])

    def call(self, inputs, training=False):
       
        return  self.model(inputs, training=training)


    @staticmethod
    def generate_noise(batch_size, noise_dim=NOISE_DIM):
        """Generate Gaussian noise for the generator input."""
        return np.random.normal(0, 1, (batch_size, noise_dim))
    


### The Discriminator

In [22]:
class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = tf.keras.Sequential([
            # First conv block - no batchnorm
            tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', 
                                input_shape=(IMG_SIZE, IMG_SIZE, 1)),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dropout(0.3),
            
            # Second conv block
            tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dropout(0.3),
            
            # Third conv block
            tf.keras.layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
            tf.keras.layers.LeakyReLU(alpha=0.2),
            tf.keras.layers.Dropout(0.3),
            
            # Output layer
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(1)  # No activation - we'll use from_logits=True in loss
        ])
    def call(self, inputs, training=False):
        return self.model(inputs, training=training)

In [26]:
class DCGAN(tf.keras.Model):
    def __init__(self, noise_dim=NOISE_DIM):
        super(DCGAN, self).__init__()
        self.noise_dim = noise_dim
        self.generator = Generator(noise_dim)
        self.discriminator = Discriminator()
        
        # We'll now use separate optimizers instead of compiling the models individually
        self.generator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
        self.discriminator_optimizer = tf.keras.optimizers.Adam(0.0002, 0.5)
        
        # Define the GAN model for inference purposes only
        self.gan = None
    
    def compile(self):
        super(DCGAN, self).compile()
        
    @tf.function
    def train_step(self, real_images, batch_size):
        # Generate noise
        
        noise = tf.random.normal([batch_size, self.noise_dim])
        
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # Generate fake images
            generated_images = self.generator(noise, training=True)
            
            # Get predictions
            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)
            
            # Calculate losses
            gen_loss = self.generator_loss(fake_output)
            disc_loss = self.discriminator_loss(real_output, fake_output)
        
        # Calculate gradients
        gradients_of_generator = gen_tape.gradient(
            gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(
            disc_loss, self.discriminator.trainable_variables)
        
        # Apply gradients
        self.generator_optimizer.apply_gradients(
            zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(
            zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        return {'d_loss': disc_loss, 'g_loss': gen_loss}
    
    def discriminator_loss(self, real_output, fake_output):
        # Use from_logits=True for numerical stability
        real_labels = tf.ones_like(real_output) * 0.9
        real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            real_labels, real_output)
        fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            tf.zeros_like(fake_output), fake_output)
        total_loss = real_loss + fake_loss
        return total_loss

    def generator_loss(self, fake_output):
        return tf.keras.losses.BinaryCrossentropy(from_logits=True)(
            tf.ones_like(fake_output), fake_output)
    def train(self, real_images, epochs, batch_size):
        # Rescale the images to [-1, 1]
        real_images = (real_images - 0.5) * 2
        
        num_batches = len(real_images) // batch_size
        
        for epoch in range(epochs):
            # Shuffle data for each epoch
            indices = np.random.permutation(len(real_images))
            shuffled_images = real_images[indices]
            
            d_loss_total, g_loss_total = 0, 0
            
            # Process full batches
            for batch_idx in range(num_batches):
                start = batch_idx * batch_size
                end = start + batch_size
                batch_images = shuffled_images[start:end]
                
                # Train on this batch
                losses = self.train_step(batch_images, batch_size)
                d_loss_total += losses['d_loss']
                g_loss_total += losses['g_loss']
            
            # Process last partial batch if needed
            if len(real_images) % batch_size != 0:
                last_batch = shuffled_images[num_batches * batch_size:]
                if len(last_batch) > 1:  # Ensure batch size > 1 for batch normalization
                    losses = self.train_step(last_batch, len(last_batch))
                    d_loss_total += losses['d_loss']
                    g_loss_total += losses['g_loss']
                    num_batches += 1  # Account for the extra batch in the average
            
            # Calculate average losses
            d_loss_avg = d_loss_total / num_batches
            g_loss_avg = g_loss_total / num_batches
            
            # Print progress
            if epoch % 5 == 0 or epoch == epochs - 1:
                print(f"Epoch {epoch + 1}/{epochs}, Discriminator Loss: {d_loss_avg:.4f}, Generator Loss: {g_loss_avg:.4f}")
                self.save_generated_images(epoch)
    
    def generate_synthetic_data(self, num_samples, num_classes):
        """Generate synthetic data with balanced labels."""
        synthetic_images, synthetic_labels = [], []
        samples_per_class = num_samples // num_classes
        
        for digit in range(num_classes):
            noise = tf.random.normal([samples_per_class, self.noise_dim])
            generated_imgs = self.generator(noise, training=False)
            generated_imgs = (generated_imgs + 1) / 2.0  # Rescale to [0, 1]
            
            synthetic_images.extend(generated_imgs.numpy())
            synthetic_labels.extend([digit] * samples_per_class)
        
        return np.array(synthetic_images), np.array(synthetic_labels)
    
    def save_generated_images(self, epoch, output_dir='generated_images'):
        """Save 16 generated images for visualization."""
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        noise = tf.random.normal([16, self.noise_dim])
        gen_imgs = self.generator(noise, training=False)
        gen_imgs = (gen_imgs + 1) / 2.0  # Rescale to [0, 1]
        
        fig, axs = plt.subplots(4, 4, figsize=(10, 10))
        cnt = 0
        for i in range(4):
            for j in range(4):
                axs[i, j].imshow(gen_imgs[cnt].numpy()[:, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        
        plt.savefig(os.path.join(output_dir, f'epoch_{epoch}.png'))
        plt.close()

In [ ]:
# Initialize the DCGAN
dcgan = DCGAN()
dcgan.compile()

# Load and process the entire training data from the generator
print("Loading and processing the entire dataset...")
all_images = []
all_labels = []

# Get the number of batches in the generator
num_batches = len(train_gen)
print(f"Processing {num_batches} batches...")

# Reset the generator to start from the beginning
train_gen.reset()

# Collect all images and labels from the generator
for i in range(num_batches):
    batch_images, batch_labels = next(train_gen)
    if batch_images.shape[0] != BATCH_SIZE:
        # Skip the last batch if it's smaller than BATCH_SIZE
        continue
    all_images.append(batch_images)
    all_labels.append(batch_labels)
    if (i+1) % 10 == 0:
        print(f"Processed {i+1}/{num_batches} batches")

# Combine all batches into single arrays
train_images = np.vstack(all_images)
train_labels = np.vstack(all_labels)

print("Training data shape:", train_images.shape)

# Reshape if needed (depending on your data's format)
if len(train_images.shape) == 3 or train_images.shape[-1] != 1:
    train_images = train_images.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print("Reshaped training data:", train_images.shape)
print("Label shape:", train_labels.shape)

# Train the DCGAN
dcgan.train(train_images, EPOCHS, BATCH_SIZE)

# Generate synthetic data
num_samples = 10000
num_classes = 10
synthetic_images, synthetic_labels = dcgan.generate_synthetic_data(num_samples, num_classes)

# Save the synthetic data
np.save('synthetic_images.npy', synthetic_images)
np.save('synthetic_labels.npy', synthetic_labels)
print(f"Generated and saved {num_samples} synthetic samples")

Loading and processing the entire dataset...
Processing 79 batches...
Processed 10/79 batches
Processed 20/79 batches
Processed 30/79 batches
Processed 40/79 batches
Processed 50/79 batches
Processed 60/79 batches
Processed 70/79 batches
Training data shape: (9984, 28, 28, 1)
Reshaped training data: (9984, 28, 28, 1)
Label shape: (78, 128)
Epoch 1/200, Discriminator Loss: 0.6865, Generator Loss: 2.2331
Epoch 6/200, Discriminator Loss: 1.2475, Generator Loss: 0.9695
Epoch 11/200, Discriminator Loss: 1.1868, Generator Loss: 1.1057
Epoch 16/200, Discriminator Loss: 1.2248, Generator Loss: 1.0378
Epoch 21/200, Discriminator Loss: 1.2152, Generator Loss: 1.0969
Epoch 26/200, Discriminator Loss: 1.1952, Generator Loss: 1.1266
Epoch 31/200, Discriminator Loss: 1.1736, Generator Loss: 1.1610
Epoch 36/200, Discriminator Loss: 1.1512, Generator Loss: 1.2005
Epoch 41/200, Discriminator Loss: 1.1507, Generator Loss: 1.2089
Epoch 46/200, Discriminator Loss: 1.1477, Generator Loss: 1.2093
Epoch 51/2